# Predicting Prices of Used Cars on Craigslist

## Impute the Data

When looking at the vehicles we note that there are lot of nulls about the vehicles features (example: the number of cylinders, drive, transmission, size, and type of vehicle). When buying a vehicle these are things you would usually ask the seller because it does effect the price of the vehicle (a vehicle with more cylinders usually is of higher value than one is of less). 

While these features can differ within a make (aka brand here listed as manufacturer) model, and model year (MMY) of a vehicle we can narrow down what the feature should be given the other features of the vehicle. For example: a  2016 Jaguar F-Type  can be all wheel drive or all-wheel drive ,  be a coupe or convertible, be automatic or manual transmission. You would think there be at least 8 configurations of the F-type.   However only a certain grouping of configurations (called Trims) that these vehicle are sold as.  There are actually on 6 configurations for the 2016 Jaguar F-Type because you can only get a manual transmission with rear wheel drive. 

If we don't have enough data to impute the vehicle given MMY and the other features we can determine the feature given its MMY. This is because A) There is usually a most popular trim for the MMY of a vehicle. B) Some features do not differ within MMY for example the 2017 Honda Accord only has 4 cylinders. 

If MMY does not give us a value we can look at just the make and model. The most likely configuration for model does change for each model year. However the trims of these features only change for every redesign of the model which does not happen every model year. So after looking a MMY we look at make model.


Load data

In [46]:
#load python packages
import os
import pandas as pd
import numpy as np

#load data
basepath = os.getcwd()
sub_file = 'data'
file = 'auto_data_m_cl.csv'
file_name_path = os.path.join(basepath, sub_file, file)
auto_data= pd.read_csv(file_name_path)


In [ ]:
#Determine how many nulls there are
auto_data.isnull().sum()

In [ ]:
#Double check data columns type and cylinders are clean

auto_data['type']= auto_data['type'].str.title().str.strip()
auto_data['cylinders'] = pd.to_numeric(auto_data['cylinders'])

In [47]:

print(auto_data['type'].value_counts())

Sedan          30180
Suv            25195
Pickup         18029
Truck          12821
Other          10044
Coupe           7996
Hatchback       6954
Van             3599
Wagon           3052
Mini-Van        2128
Convertible     1624
Offroad          156
Bus              137
Name: type, dtype: int64
id                              0
region                          0
price                           0
year                            0
manufacturer                 2837
model                           0
condition                       0
cylinders                   38128
fuel                            3
odometer                        0
title_status                    6
transmission                  130
drive                       22899
size                        82643
type                        13949
paint_color                     0
state                           0
lat                          1118
long                         1118
age                             0
mmy_a                 

### Create Trim Configuratons and MM categories

In [48]:

q1 = auto_data
#Create possible Trim configurations look at cylinders, drive and transmission
q1['mmy_a_d_t'] = q1['mmy_a'].astype(str)+' '+q1['drive'].astype(str)+' '+q1['transmission'].astype(str)
q1['mmy_a_t'] = q1['mmy_a'].astype(str)+' '+q1['transmission'].astype(str)
q1['mmy_a_d'] = q1['mmy_a'].astype(str)+' '+q1['drive'].astype(str)

#Remove trim if there is missing value in the grouping
q1.loc[(q1['transmission'].isna()), 'mmy_a_d_t' ] = np.nan
q1.loc[(q1['transmission'].isna()), 'mmy_a_t' ] = np.nan
q1.loc[(q1['transmission'].str.contains('nan', na= False)), 'mmy_a_d_t' ] = np.nan
q1.loc[(q1['transmission'].str.contains('nan', na= False)), 'mmy_a_t' ] = np.nan
q1.loc[(q1['drive'].isna()), 'mmy_a_d_t' ] = np.nan
q1.loc[(q1['drive'].isna()), 'mmy_a_d' ] = np.nan
q1.loc[(q1['drive'].str.contains('nan', na= False)), 'mmy_a_d_t' ] = np.nan
q1.loc[(q1['drive'].str.contains('nan', na= False)), 'mmy_a_d' ] = np.nan

#Create Make Model Category
q1['mm']=q1['manufacturer'].astype(str)+' '+q1['model'].astype(str)
q1['mm2']=q1['manufacturer'].astype(str)+' '+q1['model2'].astype(str)


Define functions to ease imputation coding

In [49]:
def comp_F(q1):
    """ 
    Input Auto data
    ___
    Copy columns cylinders, size, Category, type, drive and transmission 
    into new columns cylinders1, size1, Category1, type1, drive1 and transmission1
    """
    q1['cylinders1'] = q1['cylinders']
    q1['size1'] = q1['size']
    q1['Category1'] = q1['Category']
    q1['type1'] = q1['type']
    q1['drive1'] = q1['drive']
    q1['transmission1'] = q1['transmission']
    
def comp_D(q1):
    """ 
    Input Auto data
    ___
    Fill nulls from columns cylinders, size, Category, type, drive and transmission
    with the values from columns cylinders1, size1, Category1, type1, drive1 and transmission1
    """
    q1['cylinders'] = q1['cylinders'].fillna(q1['cylinders1'])
    q1['size'] = q1['size'].fillna(q1['size1'])
    q1['Category'] = q1['Category'].fillna(q1['Category1'])
    q1['type'] = q1['type'].fillna(q1['type1'])
    q1['drive'] = q1['drive'].fillna(q1['drive1'])
    q1['transmission1'] = q1['transmission'].fillna(q1['transmission1'])

#### Impute null values by Trim
Impute null values in columns MSRP, cylinders, size, Category, type, drive and transmission with mode value by Trim

In [50]:
#Generate new columns MSRP1, cylinders1, size1, Category1, type1, drive1 and transmission1
comp_F(q1)
q1['MSRP1'] = q1['MSRP']

print('Before imputation the number of nulls are:')
print(q1[['cylinders', 'size', 'type', 'drive', 'transmission', 'Category' ]].isnull().sum())

#For each trim calculate the mode of columns MSRP, cylinders, size, Category, type, drive and transmission
q1['MSRP1'] = q1.groupby(['mmy_a', 'cylinders'], sort=False)['MSRP'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['cylinders1'] = q1.groupby(['mmy_a_d_t'], sort=False)['cylinders1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['size1'] = q1.groupby(['mmy_a_d_t'], sort=False)['size1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['type1'] = q1.groupby(['mmy_a_d_t'], sort=False)['type1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['Category1'] = q1.groupby(['mmy_a_d_t'], sort=False)['Category1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['drive1'] = q1.groupby(['mmy_a_t'], sort=False)['drive1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['transmission1'] = q1.groupby(['mmy_a_d'], sort=False)['transmission1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))

#Fill nulls in columns MSRP, cylinders, size, Category, type, drive and transmission
# with values their respective calculated column 
comp_D(q1)
q1['MSRP'] = q1['MSRP'].fillna(q1['MSRP1'])


print('After trim imputation the number of nulls are:')
print(q1[['cylinders', 'size', 'type', 'drive', 'transmission', 'Category', 'MSRP' ]].isnull().sum())



Before imputation the number of nulls are:
cylinders       38128
size            82643
type            13949
drive           22899
transmission      130
Category        97038
dtype: int64
After trim imputation the number of nulls are:
cylinders        26101
size             53662
type              7062
drive             7192
transmission       130
Category         97038
MSRP            121663
dtype: int64


#### Impute null values by MMY 
Impute null values in columns MSRP, cylinders, size, Category, type, drive and transmission with mode value 
by MMY (Make, Model, Model year)

In [51]:

#Generate new columns MSRP1, cylinders1, size1, Category1, type1, drive1 and transmission1
comp_F(q1)
q1['MSRP1'] = q1['MSRP']

#For each MMY calculate the mode of columns MSRP, cylinders, size, Category, type, drive and transmission
q1['MSRP1'] = q1.groupby(['mmy_a'], sort=False)['MSRP'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['cylinders1'] = q1.groupby(['mmy_a'], sort=False)['cylinders1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['size1'] = q1.groupby(['mmy_a'], sort=False)['size1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['type1'] = q1.groupby(['mmy_a'], sort=False)['type1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['Category1'] = q1.groupby(['mmy_a'], sort=False)['Category1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['drive1'] = q1.groupby(['mmy_a'], sort=False)['drive1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['transmission1'] = q1.groupby(['mmy_a'], sort=False)['transmission1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))

#Fill nulls in columns MSRP, cylinders, size, Category, type, drive and transmission
# with values their respective calculated column 
comp_D(q1)
q1['MSRP'] = q1['MSRP'].fillna(q1['MSRP1'])

print('After manufacture model and model year  imputation the number of nulls are:')
print(q1[['cylinders', 'size', 'type', 'drive', 'transmission', 'Category' ]].isnull().sum())


After manufacture model and model year  imputation the number of nulls are:
cylinders       18777
size            40472
type             2556
drive            6389
transmission      130
Category        97038
dtype: int64


#### Impute null values by Make Model
Impute null values in columns MSRP, cylinders, size, Category, type, drive and transmission with mode value 
by MMY (Make, Model, Model year)

In [52]:
#Generate new columns cylinders1, size1, Category1, type1, drive1 and transmission1
comp_F(q1)

#For each make model calculate the mode of columns MSRP, cylinders, size, Category, type, drive and transmission
q1['cylinders1'] = q1.groupby(['mm'], sort=False)['cylinders1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['size1'] = q1.groupby(['mm'], sort=False)['size1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['type1'] = q1.groupby(['mm'], sort=False)['type1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['Category1'] = q1.groupby(['mm'], sort=False)['Category1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['drive1'] = q1.groupby(['mm'], sort=False)['drive1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))
q1['transmission1'] = q1.groupby(['mm'], sort=False)['transmission1'].apply(lambda x: x.fillna(next(iter(x.mode()), np.nan)))

#Fill nulls in columns cylinders, size, Category, type, drive and transmission
# with values their respective calculated column 
comp_D(q1)
print('After manufacture model and model year  imputation the number of nulls are:')
print(q1[['cylinders', 'size', 'type', 'drive', 'transmission', 'Category' ]].isnull().sum())


After manufacture model and model year  imputation the number of nulls are:
cylinders       12907
size            26774
type             1253
drive            3228
transmission      130
Category        95929
dtype: int64


In [53]:
q1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135864 entries, 0 to 135863
Data columns (total 42 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       135864 non-null  int64  
 1   region                   135864 non-null  object 
 2   price                    135864 non-null  int64  
 3   year                     135864 non-null  float64
 4   manufacturer             133027 non-null  object 
 5   model                    135864 non-null  object 
 6   condition                135864 non-null  object 
 7   cylinders                122957 non-null  float64
 8   fuel                     135861 non-null  object 
 9   odometer                 135864 non-null  float64
 10  title_status             135858 non-null  object 
 11  transmission             135734 non-null  object 
 12  drive                    132636 non-null  object 
 13  size                     109090 non-null  object 
 14  type

In [54]:
#Drop Excess Variables that were used for imputation
q1 = q1.drop(columns=['cylinders1', 'size1','Category1','type1' , 'drive1', 'transmission1', 'MSRP1'], axis=1 )
q1 = q1.drop(columns=['mmy_a', 'mmy_a2','mmy_a_d_t','mmy_a_d' , 'mmy_a_t', 'mm', 'mm2', 'model2'], axis=1 )

#### Impute null values by in lat, long, fuel, paint color and  MSRP

In [55]:
#impute longitude and latitude from state
q1['lat'] = q1.groupby(['state'], sort=False)['lat'].apply(lambda x: x.fillna(x.mode().iloc[0]))
q1['long'] = q1.groupby(['state'], sort=False)['long'].apply(lambda x: x.fillna(x.mode().iloc[0]))

#impute fuel
q1['fuel'] = q1['fuel'].fillna(q1['fuel'].mode()[0])
q1['MSRP'] = q1['MSRP'].fillna(0)

#If no paint color is given its unlisted and its unrealistc to try to impute it
q1['paint_color'] = q1['paint_color'].fillna('unlisted')

print(q1.isnull().sum())

id                             0
region                         0
price                          0
year                           0
manufacturer                2837
model                          0
condition                      0
cylinders                  12907
fuel                           0
odometer                       0
title_status                   6
transmission                 130
drive                       3228
size                       26774
type                        1253
paint_color                    0
state                          0
lat                            0
long                           0
age                            0
MSRP                           0
Category                   95929
State                          0
Avg_Gas_Price                  0
Division                       0
Median Household Income        0
State_Code                     0
dtype: int64


#### Create Columns Average Mileage and Residuals

In [57]:
q1['Avg_Mileage'] = q1['odometer']/ q1['age']
q1['resid'] = q1['MSRP']/ q1['price']
q1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135864 entries, 0 to 135863
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       135864 non-null  int64  
 1   region                   135864 non-null  object 
 2   price                    135864 non-null  int64  
 3   year                     135864 non-null  float64
 4   manufacturer             133027 non-null  object 
 5   model                    135864 non-null  object 
 6   condition                135864 non-null  object 
 7   cylinders                122957 non-null  float64
 8   fuel                     135864 non-null  object 
 9   odometer                 135864 non-null  float64
 10  title_status             135858 non-null  object 
 11  transmission             135734 non-null  object 
 12  drive                    132636 non-null  object 
 13  size                     109090 non-null  object 
 14  type

## Cleaning Category/ type
* Category refers to the type of design of the vehicle ( example hatchback), and it is something people take into consideration while buying a vehicle
* However the naming convention for this is non standard across manufactuers ("pickup" vs "truck" refers to the same type from Ford and GM respectively)
* Considering there are many users on Craigslist even though type is often in the description of a vehicle sold the naming convention would not be consistent from one user description to the next. 
* Since Category type is scraped from the user description means that type is inconsistent across the database

* A Singular third party from an automible expert in this case the USA government will give us a consistent naming convention across all vehicles
* However the goverment gives us a group of possible types a vehicle could be given make model model year

* By combining the user description and the third party categories we get a specific category for each vehicle which has a conistent naming convention across vehicles

* For the merge's we have we are going to check if the category within the description matches what the third party gave


Check to see unique values in type and Category

In [ ]:
q1['type'].value_counts()

In [ ]:
q1['Category'].value_counts()

Transform Category Column and Type Column

In [ ]:
#Transform empty strings in Category into null values
q1['Category']= q1['Category'].replace(r'^\s*$', np.nan, regex=True)

#Create column that has the first possible category vehicle could be
q1['Category2']= q1['Category'].str.strip().str.split(', ').str[0]

#Create column that has the number of possible categories a vehicle could be
q1['Category3']= q1['Category'].str.strip().str.split(', ')
q1['Category3']= q1['Category3'].str.len()


#In type column make truck and pick up consistent
q1['type'] = q1['type'].replace('Truck', 'Pickup')

#Create column with type where Other is listed as null and Van/ Mini-Van and Van/Minivan are all Can/Minivan
q1['type2']= q1['type2'].replace('Other', np.nan)
q1['type2'] = q1['type2'].replace('Mini-Van', 'Van')
q1['type2'] = q1['type2'].replace('Van', 'Van/Minivan')

In [ ]:
#To iterate over variable we cannot have values so we fill na with blank value
q1['Categoryf']= q1['Category'].fillna('blank')
q1['typef']= q1['type2'].fillna('blank')

#Create Column to put cleaned Category variable into
q1['Category_Cleaned']= np.nan



In [ ]:
#If category length = 1 then Category_Cleaned= Category variable
q1['Category_Cleaned'] = np.where(q1['Category3'] == 1 , q1['Category'], np.nan)
print(q1['Category_Cleaned'].isnull().sum())

#If the value in the type column is in the list within the Category column then Cleaned_Category=type
#check if vehicle type listed in description is in Category Column
q1['cat_tiki_check0']= q1.apply(lambda x: x.typef in x.Categoryf, axis=1)
#Replace Category_Cleaned variable with type variable if check is true
q1.loc[q1['cat_tiki_check0'] == True, 'Category_Cleaned'] = q1.type
print(q1['Category_Cleaned'].isnull().sum())

#If category length = 2 we pick the first category
q1['Category_Cleaned1']= np.nan
q1['Category_Cleaned1'] = np.where(q1['Category3'] == 2 , q1['Category2'], np.nan)
q1['Category_Cleaned'] = q1['Category_Cleaned'].fillna(q1['Category_Cleaned1'])
q1['Category_Cleaned'] = q1['Category_Cleaned'].fillna('Unlisted')

q1 =q1.drop(columns = ['Category2', 'type2', 'Category3', 'Categoryf', 'typef', 'cat_tiki_check0', 'Category_Cleaned1'] )


#### Turn all string variables into Categorical columns

In [58]:

categorical_columns = q1.select_dtypes(exclude=['int64', 'float']).columns.tolist()

for col in categorical_columns:
    q1[col] = q1[col].astype('category')
q1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135864 entries, 0 to 135863
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   id                       135864 non-null  int64   
 1   region                   135864 non-null  category
 2   price                    135864 non-null  int64   
 3   year                     135864 non-null  float64 
 4   manufacturer             133027 non-null  category
 5   model                    135864 non-null  category
 6   condition                135864 non-null  category
 7   cylinders                122957 non-null  float64 
 8   fuel                     135864 non-null  category
 9   odometer                 135864 non-null  float64 
 10  title_status             135858 non-null  category
 11  transmission             135734 non-null  category
 12  drive                    132636 non-null  category
 13  size                     109090 non-null  ca

Save Data

In [59]:
#save data

new_file = 'auto_data_merged_cl.csv'
new_file_name_path = os.path.join(basepath,sub_file, new_file)
q1.to_csv(new_file_name_path, index=False)